In [1]:
import openpyxl
import pandas as pd
import numpy as np

In [3]:
turbine_data = pd.read_csv('C:/Users/mprillim/sam_dev/sam/deploy/libraries/Wind Turbines.csv') #read in old turbine data
#print(turbine_data)
#Load in turbine data modifications from Excel workbook
new_data = openpyxl.load_workbook(filename= "C:/Users/mprillim/sam_dev/sam/deploy/libraries/new_wind_data.xlsx",data_only=True)
#Read data from first tab in workbook, which contains a summary of current SAM library along with modified rotor diameter measurements and new suggested turbine technologies
new_wb_main = new_data['Wind Turbines'] 
#Convert Wind Turbines data into a pandas DataFrame for easier data handling
new_wb_main_df = pd.DataFrame(new_wb_main.values,columns=['ID','Name','Model Name','KW Rating','Nameplate Capacity','Rated Capacity','Rotor Diameter','Corrected Rotor Diameter','Commercial Available','Certified','Date of Certification','Power Curve Source','Duplicate','Notes','IEC Wind Speed Class','Wind Speed Array','Power Curve Array'])
#print(new_wb_main_df)
#old_names = turbine_data['Name']
#print(old_names)
#print(new_wb_main_df.iloc[65,:])
#print((turbine_data.iloc[65,:]))



In [4]:
for r in turbine_data.index:
    if r > 1:
        sam_name = (turbine_data.iloc[r,0])
        row = new_wb_main_df[new_wb_main_df.iloc[:,1]==sam_name]
        print(row)
        turbine_data.iloc[r,2]= row.iloc[0,7]
        print(r)
print(turbine_data)

  ID                   Name      Model Name KW Rating Nameplate Capacity  \
1  1  Ampair 600-230 1.7m    Ampair 600-230      0.27               None   

  Rated Capacity Rotor Diameter Corrected Rotor Diameter Commercial Available  \
1           None              1                      1.7                   No   

  Certified Date of Certification Power Curve Source Duplicate Notes  \
1        No                   N/A               None      None  None   

  IEC Wind Speed Class                                   Wind Speed Array  \
1              unknown  1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|1...   

                                   Power Curve Array  
1  0|0|0|0.004|0.035|0.072|0.113|0.151|0.184|0.21...  
2
  ID                          Name              Model Name KW Rating  \
2  2  Samrey Samprey Wren_1m_0.3kw  Samrey  Wren 1m 0.3 kW      0.28   

  Nameplate Capacity Rated Capacity Rotor Diameter Corrected Rotor Diameter  \
2               None           None             

44  0|0|0|0.12|0.27|0.48|0.8|1.15|1.56|2.01|2.5|2....  
45
    ID                Name     Model Name KW Rating Nameplate Capacity  \
45  45  ReDriven 3.8m 3kw   ReDriven 3.8m     3.327               None   
46  46  ReDriven 3.8m 3kw   ReDriven 3.8m     3.327               None   

   Rated Capacity Rotor Diameter Corrected Rotor Diameter  \
45           None              3                      3.8   
46           None              3                      3.8   

   Commercial Available Certified Date of Certification Power Curve Source  \
45                   No        No                   N/A               None   
46                   No        No                   N/A               None   

   Duplicate Notes IEC Wind Speed Class  \
45      None  None              unknown   
46      None  None              unknown   

                                     Wind Speed Array  \
45  1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|1...   
46  1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|1...  

90  0|0|0.51|1.3|2.95|5.55|8.5|12.65|17.48|20.24|2...  
91
    ID                         Name   Model Name KW Rating Nameplate Capacity  \
91  91  Coemi Skywing G-3 20kW 15m   Skywing G-3      21.5               None   

   Rated Capacity Rotor Diameter Corrected Rotor Diameter  \
91           None             15                       15   

   Commercial Available Certified Date of Certification Power Curve Source  \
91                   No        No                   N/A               None   

   Duplicate Notes IEC Wind Speed Class  \
91      None  None              unknown   

                                     Wind Speed Array  \
91  1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|1...   

                                    Power Curve Array  
91  0|0|0|2.8|4.3|5.9|7.5|9.9|12.7|15.9|19|20.9|21...  
92
    ID                  Name      Model Name KW Rating Nameplate Capacity  \
92  92  ReDriven 12.3m 20kw   ReDriven 12.3m    22.159               None   
93  93  ReDriven 12.3m 20kw

IndexError: single positional indexer is out-of-bounds

In [5]:
#Merge existing turbine library and new library data on the basis of the matching Name, kW rating, Rotor Diameter, IEC Wind speed class variables
new_rotor_df = pd.merge(turbine_data,new_wb_main_df,how='right',on=['Name','KW Rating', 'Rotor Diameter', 'IEC Wind Speed Class', 'Wind Speed Array','Power Curve Array'])
#print(new_rotor_df)
#Find indices where merged dataframe does not have a Name property, as these are the newly introduced turbine technologies
new_turbine_check = new_rotor_df.index[new_rotor_df['Name'].isnull()]
#print(new_turbine_check)
#new_rotor_df.iloc[283:,[0,1,10]] = new_rotor_df.iloc[283:,[7,8,2]]
#print(new_rotor_df.iloc[:,10])
#Separate 6 columns of SAM turbine library into new dataframe
new_wave_library = pd.DataFrame(new_rotor_df.iloc[:,[0,10,1,3,4,5]],columns=['Name','KW Rating','Corrected Rotor Diameter','IEC Wind Speed Class','Wind Speed Array','Power Curve Array'])
#Assign Name values of new turbine technologies from Name column of new library worksheet
new_wave_library.iloc[new_turbine_check,0]=new_rotor_df.iloc[new_turbine_check,7]
#Assign kW rating values of new turbines
new_wave_library.iloc[new_turbine_check,1]=new_rotor_df.iloc[new_turbine_check,8]
#Assign Corrected rotor diameter of new turbines
new_wave_library.iloc[new_turbine_check,2]=new_rotor_df.iloc[new_turbine_check,10]
#Assign unknown IEC wind class for new turbines (not listed in modification list)
new_wave_library.iloc[new_turbine_check,3]='unknown'
#Assign empty wind speed arrays for new turbines to be filled from next tabs in modification workbook
new_wave_library.iloc[new_turbine_check,4]=new_rotor_df.iloc[new_turbine_check,4]
#Assign empty power curve arrays for new turbines to be filled from next tabs in modification workbook
new_wave_library.iloc[new_turbine_check,5]=new_rotor_df.iloc[new_turbine_check,5]
#print(new_wave_library.iloc[121:126,:])
#print(new_rotor_df.iloc[121:126,[7,8,10,3,4,5]])
#new_wave_library = new_wave_library.drop(new_wave_library.index[0])
new_wave_library = new_wave_library.rename(columns={'Corrected Rotor Diameter': 'Rotor Diameter'})
new2 = new_wave_library.rename(columns={'Corrected Rotor Diameter': 'Rotor Diameter'})
#print(new2)
new2.to_csv('C:/Users/mprillim/sam_dev/sam/deploy/libraries/wind_library_test.csv')

In [8]:
#Remaining tabs of the workbook each contain wind speed and power curve arrays that have changed from existing SAM libraries or for new turbine technologies to be added into SAM
#Store tab names of the modification workbook
sheet_names = pd.DataFrame(new_data.sheetnames)
#print(sheet_names.iloc[0,0])
#Create empty dataframe to be filled with new array data, names of worksheet tabs
check_array= pd.DataFrame([],index=new_wave_library.index)
check_array['Name Check']=np.nan #Name column to be filled with tab names
check_array['New Wind Speed Array']=np.nan #Wind speed array string column to be filled through iteration
check_array['New Power Curve Array']=np.nan #Power curve array string column to be filled through iteration
worksheet_loop = np.linspace(1,sheet_names.index[-6],11,endpoint=True) #Loop through first 11 tabs for modified turbine rating data
#print(worksheet_loop)
for j in worksheet_loop:
    b = int(j) #integer of iterator j
    #print(b)
    sheet_data = sheet_names.iloc[b,0] #get tab name at index position

    #print(sheet_data)
    ws = new_data[str(sheet_data)] #Worksheet data based on tab name
    ws_df = pd.DataFrame(ws.values) #convert worksheet data to dataframe for easier data handling
    if isinstance(ws_df.iloc[5,0],str)==True: #check which row of worksheet data starts (not consistent)
        start_var = 6
    else:
        start_var = 5
    #print(ws_df)
    loop_array = np.linspace(start_var,ws_df.index[-1]+1,ws_df.index[-1]-start_var+1,endpoint=False) #Loop through indices of worksheet to build arrays
    print(loop_array)
    for i in loop_array:
            a = int(i) #integer of iterator i
            #print(a)
            if ws_df.iloc[a,0] is None: #Check each array (old and new, wind and power) for nonetype (empty cells)
                ws_df.iloc[a,0]=9999 #replace nonetype values with 9999 for number formatting purposes
            if ws_df.iloc[a,5] is None:
                ws_df.iloc[a,5]=9999
            if ws_df.iloc[a,1] is None:
                ws_df.iloc[a,1]=9999    
            if ws_df.iloc[a,6] is None:
                ws_df.iloc[a,6]=9999
            if ws_df.iloc[a,6] < 0:
                ws_df.iloc[a,6] = 0;
            if i==start_var: #first row of array, print number followed by separator |
                old_wind_speed_string = str(ws_df.iloc[a,0])+"|" #ensure 3 digits after the decimal (eliminate hanging zeros)
                new_wind_speed_string = str('%5.2f' % ws_df.iloc[a,5])+"|"
                old_power_curve_string = str(ws_df.iloc[a,1])+"|"
                new_power_curve_string = str('%5.3f' % ws_df.iloc[a,6])+"|"
            elif i==ws_df.index[-1]: #Last row of array, do not add trailing separator |
                old_wind_speed_string = old_wind_speed_string + str(ws_df.iloc[a,0])
                new_wind_speed_string = new_wind_speed_string + str('%5.2f' % ws_df.iloc[a,5])
                old_power_curve_string = old_power_curve_string + str(ws_df.iloc[a,1])
                new_power_curve_string = new_power_curve_string + str('%5.3f' % ws_df.iloc[a,6])
            else: #Middle rows, append next row's data to previous index followed by separator |
                old_wind_speed_string = old_wind_speed_string + str(ws_df.iloc[a,0])+"|"
                new_wind_speed_string = new_wind_speed_string + str('%5.2f' % ws_df.iloc[a,5])+"|"
                old_power_curve_string = old_power_curve_string + str(ws_df.iloc[a,1])+"|"
                new_power_curve_string = new_power_curve_string + str('%5.3f' % ws_df.iloc[a,6])+"|"
            #remove nonetypes from array strings
            old_wind_speed_string = old_wind_speed_string.replace('9999|',"") 
            new_wind_speed_string = new_wind_speed_string.replace('9999.00|',"")
            old_power_curve_string = old_power_curve_string.replace('9999|',"")
            new_power_curve_string = new_power_curve_string.replace('9999.000|',"")
            old_wind_speed_string = old_wind_speed_string.replace('|9999',"")
            new_wind_speed_string = new_wind_speed_string.replace('|9999.00',"")
            old_power_curve_string = old_power_curve_string.replace('|9999',"")
            new_power_curve_string = new_power_curve_string.replace('|9999.000',"")
            old_wind_speed_string = old_wind_speed_string.replace(" ","")
            new_wind_speed_string = new_wind_speed_string.replace(" ","")
            old_power_curve_string = old_power_curve_string.replace(" ","")
            new_power_curve_string = new_power_curve_string.replace(" ","")
            print(new_power_curve_string)
            print(old_power_curve_string)
            print(new_wind_speed_string)
    #Find rows of wave library where built power curve array matches listed power array
    check_array_values = new_wave_library[new_wave_library['Power Curve Array']==old_power_curve_string]# & new_wave_library['Power_Curve_Array'].str.contains[old_power_curve_string]]
    check_array.iloc[check_array_values.index,0]=str(sheet_data) #Use indices of matching data to fill tab name  
    check_array.iloc[check_array_values.index,1]=new_wind_speed_string #Use indices of matching data to fill wind speed array
    check_array.iloc[check_array_values.index,2]=new_power_curve_string #Use indices of matching data to fill power curve array
#print(old_wind_speed_string)
#print(new_wind_speed_string)
#print(new_power_curve_string)
#print(new_wave_library['Power Curve Array'])
#print(old_power_curve_string)
#print(check_array)

#print(new_wave_library)
#print(new_wind_speed_string)
#print(ws_df)
#for i in sheet_names.index:
 #   sheet_data = sheet_names.iloc[i,0]
  #  ws = new_data[str(sheet_data)]
   # ws_df = pd.DataFrame(ws.values)
    

[ 5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22.
 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37.]
0.000|
0|
0.56|
0.000|0.000|
0|0|
0.56|1.04|
0.000|0.000|0.000|
0|0|0|
0.56|1.04|1.52|
0.000|0.000|0.000|0.000|
0|0|0|0.084|
0.56|1.04|1.52|2.02|
0.000|0.000|0.000|0.000|0.000|
0|0|0|0.084|0.203|
0.56|1.04|1.52|2.02|2.51|
0.000|0.000|0.000|0.000|0.000|0.003|
0|0|0|0.084|0.203|0.391|
0.56|1.04|1.52|2.02|2.51|3.00|
0.000|0.000|0.000|0.000|0.000|0.003|0.034|
0|0|0|0.084|0.203|0.391|0.643|
0.56|1.04|1.52|2.02|2.51|3.00|3.51|
0.000|0.000|0.000|0.000|0.000|0.003|0.034|0.084|
0|0|0|0.084|0.203|0.391|0.643|0.968|
0.56|1.04|1.52|2.02|2.51|3.00|3.51|4.03|
0.000|0.000|0.000|0.000|0.000|0.003|0.034|0.084|0.132|
0|0|0|0.084|0.203|0.391|0.643|0.968|1.333|
0.56|1.04|1.52|2.02|2.51|3.00|3.51|4.03|4.46|
0.000|0.000|0.000|0.000|0.000|0.003|0.034|0.084|0.132|0.203|
0|0|0|0.084|0.203|0.391|0.643|0.968|1.333|1.748|
0.56|1.04|1.52|2.02|2.51|3.00|3.51|4.03|4.46|5.00|
0.000|0.

In [9]:
#New Turbine Power arrays input in different format than modified array values
#Loop through remaining tabs of workbook to build new turbine technology performance curves
worksheet_loop2 = np.linspace(sheet_names.index[-5],sheet_names.index[-1],5,endpoint=True)
print(worksheet_loop2)
for k in worksheet_loop2:
    c = int(k) #integer for iterator k
    print(c)
    sheet_data = sheet_names.iloc[c,0] #pull worksheet tab names

    #print(sheet_data)
    ws = new_data[str(sheet_data)] #Pull data based on worksheet tab names
    ws_df = pd.DataFrame(ws.values) #Convert data to Dataframe for easier data handling
    #print(ws_df)
    if isinstance(ws_df.iloc[5,0],str)==True: #check which row data starts on (not consistent)
        start_var=6
    else:
        start_var=5
    loop_array2 = np.linspace(start_var,ws_df.index[-1]+1,ws_df.index[-1]-start_var+1,endpoint=False) #iterate through 
    for m in loop_array2:
            d = int(m)
            #print(d)
            column2_bool = (ws_df.iloc[:,2].str.contains('Hub Height')) #column where wind speed data is located (not consistent)
            #column3_bool = (ws_df['3'].str.contains('Hub Height'))
            if any(column2_bool==True):
                if ws_df.iloc[d,2] is None: #replace nonetype values for number formattting purposes
                    ws_df.iloc[d,2]=999
                if ws_df.iloc[d,3] is None:
                    ws_df.iloc[d,3]=999
                if ws_df.iloc[d,3] < 0:
                    ws_df.iloc[d,3]= 0
                if m==start_var: #first row, start string of data followed by separator |
                    new_wind_speed_string = str('%5.2f' % ws_df.iloc[d,2])+"|"
                    new_power_curve_string = str('%5.3f' % ws_df.iloc[d,3])+"|"
                elif m==ws_df.index[-1]: #last row, finish string of data, do not add trailing separator |
                    new_wind_speed_string = new_wind_speed_string + str('%5.2f' % ws_df.iloc[d,2])
                    new_power_curve_string = new_power_curve_string + str('%5.3f' % ws_df.iloc[d,3])
                else: #middle rows, append next rows to string followed by separator |
                    new_wind_speed_string = new_wind_speed_string + str('%5.2f' % ws_df.iloc[d,2])+"|"
                    new_power_curve_string = new_power_curve_string + str('%5.3f' % ws_df.iloc[d,3])+"|"
            else: #Same iteration, but if data starts in different column
                if ws_df.iloc[d,1] is None:
                    ws_df.iloc[d,1]=999
                if ws_df.iloc[d,2] is None:
                    ws_df.iloc[d,2]=999
                if ws_df.iloc[d,2] < 0:
                    ws_df.iloc[d,2] = 0
                if m==start_var:
                    new_wind_speed_string = str('%5.2f' % ws_df.iloc[d,1])+"|"
                    new_power_curve_string = str('%5.3f' % ws_df.iloc[d,2])+"|"
                elif m==ws_df.index[-1]:
                    new_wind_speed_string = new_wind_speed_string + str('%5.2f' % ws_df.iloc[d,1])
                    new_power_curve_string = new_power_curve_string + str('%5.3f' % ws_df.iloc[d,2])
                else:
                    new_wind_speed_string = new_wind_speed_string + str('%5.2f' % ws_df.iloc[d,1])+"|"
                    new_power_curve_string = new_power_curve_string + str('%5.3f' % ws_df.iloc[d,2])+"|"
            #Remove nonetype values from array strings
            new_wind_speed_string = new_wind_speed_string.replace('999.00|',"")
            new_power_curve_string = new_power_curve_string.replace('999.000|',"")
            new_wind_speed_string = new_wind_speed_string.replace('|999.00',"")
            new_power_curve_string = new_power_curve_string.replace('|999.000',"")
            new_wind_speed_string = new_wind_speed_string.replace(" ","")
            new_power_curve_string = new_power_curve_string.replace(" ","")
    #Compare name of tab for new turbine technologies to the names in the library list
    check_array_values = new_wave_library[new_wave_library['Name'].str.contains(sheet_data)]# & new_wave_library['Power_Curve_Array'].str.contains[old_power_curve_string]]
    check_array.iloc[check_array_values.index,0]=str(sheet_data) #Fill in tab name
    check_array.iloc[check_array_values.index,1]=new_wind_speed_string #Fill in wind speed array data
    check_array.iloc[check_array_values.index,2]=new_power_curve_string #Fill in power curve array data
new_wave_library['Name Check']=check_array['Name Check'] #add Name check column to library dataframe
new_wave_library['New Wind Speed Array']=check_array['New Wind Speed Array'] #add wind speed array column to library dataframe
new_wave_library['New Power Curve Array']=check_array['New Power Curve Array'] #add power curve array column to library dataframe
#new_wave_library.to_csv('C:/Users/mprillim/sam_dev/sam/deploy/libraries/wind_library_test2.csv')

[12. 13. 14. 15. 16.]
12
13
14
15
16


In [10]:
modified_list = new_wave_library.index[~new_wave_library['New Wind Speed Array'].isnull()] #find rows where there is data for the modified wind speed array (rest na)
#print(new_wave_library)
new_wave_library.iloc[modified_list,4]=new_wave_library.iloc[modified_list,7] #where there is new wind speed data, replace array in existing SAM library
new_wave_library.iloc[modified_list,5]=new_wave_library.iloc[modified_list,8] #where there is new power curve data, replace array in existing SAM library
#new_wave_library = new_wave_library.drop(new_wave_library.index[0])
final_wind_turbine_library = new_wave_library.iloc[:,0:6]
final_wind_turbine_library.to_csv('C:/Users/mprillim/sam_dev/sam/deploy/libraries/wind_library_final.csv',index=False)
print(ws_df)

             0                                                  1  \
0      Source:                                SWCC Summary Report   
1         None  As of February 2020 http://smallwindcertificat...   
2         None                                    PNNL has a copy   
3         None                                               None   
4         None                                          CERTIFIED   
5   Bin Number                        Hub Height Wind Speed (m/s)   
6            1                                               1.52   
7            2                                                  2   
8            3                                               2.56   
9            4                                               3.02   
10           5                                               3.53   
11           6                                                  4   
12           7                                                4.5   
13           8                    

In [ ]:
sheet_data = sheet_names.iloc[2,0]
print(check_array.iloc[30,:])
#print(sheet_data)
ws = new_data[str(sheet_data)]
ws_df2 = pd.DataFrame(ws.values)
print(ws_df2)
print(str('5.3f' % ws_df.iloc[7,3]))